In [1]:
import pandas as pd
import os
import sys
from datetime import datetime
import csv
import concurrent.futures
import subprocess
from datetime import datetime

Dataset of work "SZZ in the time of Pull Requests" 
- SOURCE: https://github.com/mozilla/regressors-regressions-dataset

In [2]:
# Clone Repository
# !hg clone https://hg.mozilla.org/mozilla-central/

In [3]:
# Download dataset
# !wget https://raw.githubusercontent.com/mozilla/regressors-regressions-dataset/main/dataset.csv

In [4]:
def getRevisionId(commit_hash):
    return subprocess.getoutput("cd firefox-source && hg id -n -r "+commit_hash)

In [5]:
def calculateDistanceCommits(bfc,bic):
    aux= bic + "::" + bfc
    distance = subprocess.getoutput("cd firefox-source && hg log --rev "+aux+" --template \"{rev} \" | wc -w")
    return int(distance)

In [6]:
def getDate(c_id):
    date = subprocess.getoutput("cd firefox-source && hg log -r "+c_id+" --template \"{date|date}\n\"")
    return datetime.strptime(date, "%a %b %d %H:%M:%S %Y %z")
def calculateDistanceDays(bfc_id,bic_id):
    bfc_date = getDate(bfc_id)
    bic_date = getDate(bic_id)
    return (bfc_date-bic_date).days

In [7]:
def addDistances(link):
    bfc = link['FIX_COMMITS_MERCURIAL'].split(" ")[0]
    bfc_id = getRevisionId(bfc)
    if isinstance(link['BUG_COMMITS_MERCURIAL'], str):
        bics = link['BUG_COMMITS_MERCURIAL'].split(" ")
        commits_sum = 0
        days_sum = 0
        for bic in bics:
            bic_id = getRevisionId(bic)
            commits_sum += calculateDistanceCommits(bfc, bic)
            days_sum += calculateDistanceDays(bfc_id,bic_id)
        link['DISTANCE_COMMITS'] = commits_sum / len(bics)
        link['DISTANCE_DAYS'] = days_sum / len(bics)
        link['CALCULABLE_DISTANCE'] = True
    else:
        link['CALCULABLE_DISTANCE'] = False
        return

In [8]:
df = pd.read_csv('dataset.csv')
links = df[df['NO_BUG']==False].to_dict('records')
filtered_links = []
links_with_more_tha_one_bfc=0
for link in links:
    bfcs = link['FIX_COMMITS_GIT'].split(" ")
    if len(bfcs) > 1:
        links_with_more_tha_one_bfc+=1
        continue
    filtered_links.append(link)
len(filtered_links)

4146

In [9]:
len(df[df['NO_BUG']==True])

45

In [10]:
links_with_more_tha_one_bfc

1157

In [11]:
future_results = []
with concurrent.futures.ThreadPoolExecutor(64) as executor:
    for link in filtered_links:
        future = executor.submit(addDistances, link)
        future_results.append(future)
for future in future_results:
    try:
        future.result() 
    except Exception as e:
        print(e)

In [12]:
df = pd.DataFrame.from_records(filtered_links)
df[['DISTANCE_COMMITS','DISTANCE_DAYS']].describe().T

,count,mean,std,min,25%,50%,75%,max
DISTANCE_COMMITS,4146.0,14254.044058,39993.010694,0.0,664.0,2348.5,10171.886364,514394.0
DISTANCE_DAYS,4146.0,97.319502,316.922457,-647.0,4.5,15.0,68.000000,9195.5


In [26]:
df[df['DISTANCE_DAYS'] >= 0]

,FIX_ID,FIX_COMMITS_MERCURIAL,FIX_COMMITS_GIT,BUG_ID,BUG_COMMITS_MERCURIAL,BUG_COMMITS_GIT,NO_FILE_SHARED_FILES,NEW_LINES_ONLY_FIX,REMOVE_LINES_ONLY_BUG,NO_BUG,UNLINKABLE,DISTANCE_COMMITS,DISTANCE_DAYS,CALCULABLE_DISTANCE
0,724457,31aea004045a5572adb1a665fca15f19531b4a42,830f666746d9096b664f9cd7dc00b182011bd57d,610902,e35dfafeee4222dd02fb04c192dcd10eb24cc129,b44beb2f50adea62fb1d4c44b88c1231cd93efae,False,False,False,False,False,378769.000000,2690.000000,True
1,992384,98575a5ca794cb67b8d569e8a95188a30532e336,a3329bf3430074eaf2d41dc1b74c95019bb59d6b,1102175,004a0cb8af9fbbe84522d0506a1fdd132d51944c c4285...,d5fdf5222cfcf502466759410a076018e3220114 18826...,False,False,False,False,False,30685.444444,200.777778,True
2,1046479,d15a1a637b666008ecaa68509ea4b3c48b0b164d,8bd541c1e3cc4d447ac140a3e3bc29c32fc02a68,726781,ff57a7b6f16731ebb71d930421c8d8f4feca45e1,e2f138cb03dd3cabc0e5870d3896790416b5c306,True,False,False,False,True,286507.000000,1849.000000,True
3,1221889,e366c310a5eb2204c77288c2450727aacef5d773,ef2909de413dfb64cd4d68ae2c4958bcc09f0e4a,1477205 1205558,f1f82e673a3490bb572c3a1110eeded81badb9a9 d89b9...,ca25e7dbb6892b3ce6cd364277e6d8fe5fc60c95 89463...,False,False,False,False,False,208120.833333,1389.500000,True
4,1224848,606676e2dd214e37cee9bae611072918cbd87df7,eec931567c3e973d10701d0c6b090a4e29a0de3c,786674,f5c7c5cd504a912dd8bd1e5775c5accb97fc7a32,d11f3d18f453b646ae5619440d0a83cee726beae,False,False,False,False,False,475.000000,2.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141,1676414,2d93ec1fdb2517df6c92bf83ab66d0ca303da668,ea186e67796833032f0ed8dffe61e1171cd49783,1675549,ab2514a6da6491cffeca1445b1914826975d3395,39ed8503d344b2628e07f750fc1fef1e6cbc0fa4,False,False,False,False,False,147.000000,0.000000,True
4142,1676417,d3281939949e31fc36141751fc90206701a6c78f,37f3e27c37ad95222e17095e73713d6d7e36cd9a,1674515,5d8fba30a3f4b2550e26bc55029bb7a66dbd7484 ca520...,beeb9bf6e2990d82f01ae198f1c83bc3a2187164 bd2dd...,False,False,False,False,False,148.500000,0.500000,True
4143,1617111,2bb5b1e1caf647c617838c455527a4b7fa25431b,2de30cc56350c503a7b0b1f427a8ebb7bffd5005,1615607,95fe1470b3718d04e5c45605faedda62da140071,a019e5acf2b9584e512f47ff3c5d3369e6cabe4d,False,False,False,False,False,4921.000000,28.000000,True
4144,1623938,7172c878c66e6b997d823ce37c6f9491e856bbae,704a04f8bf2f119b40a0881d8273309eb3b149e1,1606036,dbeea4db8edeac1d346210eccbd1ed0240d5f65f,0bfa1c9de00e2d534a9f97f3427ced361c1c3702,False,False,False,False,False,11745.000000,82.000000,True


In [27]:
# There are 60 commits (from 4,146) where BFC was created BEFORE BIC (???)
len(df[df['DISTANCE_DAYS'] < 0])

90

In [28]:
len(df[df['DISTANCE_DAYS'] >= 0][df['DISTANCE_COMMITS'] == 0])

/tmp/ipykernel_610162/3162804860.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df[df['DISTANCE_DAYS'] >= 0][df['DISTANCE_COMMITS'] == 0])


0

In [29]:
df[df['DISTANCE_DAYS'] >= 0][df['DISTANCE_COMMITS'] > 0][['DISTANCE_COMMITS','DISTANCE_DAYS']].describe().T

/tmp/ipykernel_610162/3476698939.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['DISTANCE_DAYS'] >= 0][df['DISTANCE_COMMITS'] > 0][['DISTANCE_COMMITS','DISTANCE_DAYS']].describe().T


,count,mean,std,min,25%,50%,75%,max
DISTANCE_COMMITS,4056.0,14555.368756,40382.311435,4.0,697.5,2442.0,10450.875000,514394.0
DISTANCE_DAYS,4056.0,100.327986,319.483159,0.0,5.0,16.5,70.035714,9195.5


In [31]:
multiples_bfc=0
multiples_bic=0
multiples_bfc_bic=0
for link in links:
    n_bfc = link['FIX_COMMITS_GIT'].split(" ")
    n_bic = link['BUG_COMMITS_GIT'].split(" ")
    if len(n_bfc) > 1:
        multiples_bfc+=1
    if len(n_bic) > 1:
        multiples_bic+=1
    if len(n_bfc) > 1 and len(n_bic)>1:
        multiples_bfc_bic+=1
        
print("multiples_bfc=",multiples_bfc)
print("multiples_bic=",multiples_bic)
print("multiples_bfc_bic=",multiples_bfc_bic)

multiples_bfc= 1157
multiples_bic= 3396
multiples_bfc_bic= 805
